# Road Traffic Vehicle Counting Exam

In this exam, you will use the YOLO (You Only Look Once) object detection model to analyze a video of road traffic. Your primary task is to count the number of vehicles exiting the road at each of the four exits (located at the top, bottom, left, and right of the frame). The video for this task is provided at `Datasets/YOLO_Exam_Video.mp4`.

## Objectives
- Load and process the video using OpenCV.
- Utilize the YOLO model to detect vehicles in each frame.
- Manipulate the video frames using OpenCV.
- Track vehicles as they move across the video frames.
- Count the number of vehicles exiting via the top street.
- Count the number of vehicles exiting via the bottom street.
- Count the number of vehicles exiting via the left street.
- Count the number of vehicles exiting via the right street.


## Setup Environment

Before you begin, ensure you have the necessary libraries installed. You will need `opencv`, and `ultralytics` among others.
If these are not installed, you should install them.

In [1]:
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 kB 901.9 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 872.3/872.3 kB 11.8 MB/s eta 0:00:00


In [2]:
from ultralytics import YOLO
import os
import matplotlib.pyplot as plt
from IPython.display import Image, display
from IPython import display
display.clear_output()

In [3]:
import cv2
from ultralytics import YOLO

In [7]:
# Open the input video
video_path = '/content/YOLO_Video.mp4'
cap = cv2.VideoCapture(video_path)

In [8]:
width = int(cap.get(3))
height = int(cap.get(4))
fps = int(cap.get(5))

In [9]:
output_path = '/content/output_video.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

## Load the YOLO Model

In [10]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


You will first need to load the YOLO model, there is a custom model for this exam called `YOLO_Model.pt` under Datasets file ⚠️ PLEASE USE THIS MODEL AND DO NOT USE ANY OTHER MODEL ⚠️.
Write the code to load the YOLO model below:

In [11]:
model = YOLO('/content/drive/MyDrive/TT5/YOLO_Model.pt')

In [12]:
print("Model loaded successfully!")

Model loaded successfully!


## Prepare the Video Capture

Create a variable to capture the video frames, you can use `cv2.VideoCapture()` to achive this.

In [13]:
# Open the input video
video_path = '/content/YOLO_Video.mp4'
cap = cv2.VideoCapture(video_path)

## Get Video Information

You can use `cv2` library to get these information fro the `VideoCapture()` variable you created to extract these information:
* `height`: Video's height.
* `width`: Video's width.
* `fps`: Video's frames.

In [14]:
import cv2
from ultralytics import YOLO

In [15]:
# Open the input video
video_path = '/content/YOLO_Video.mp4'
cap = cv2.VideoCapture(video_path)

In [16]:
width = int(cap.get(3))
height = int(cap.get(4))
fps = int(cap.get(5))

In [17]:
output_path = '/content/output_video.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

## Prepare Video Writer to Store the Output

Create a variable that uses `cv2.VideoCapture()` to save the video with the bounding boxes, specified Region of Interest rectangle (ROI) and the counted vehicles on each sides. You will need to make the video with the same `fps`, `width`, `height`, and specify the codec and output path of the video.

In [18]:
output_path = '/content/drive/MyDrive/TT5/YOLO_Video.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

## Expermint to get the `(x1, y1, x2, y2)` of the ROI

### Hint
When defining the Regions of Interest (ROIs) for detecting vehicles exiting the road, consider the coordinates that outline the relevant areas. These coordinates can be specified as `(x1, y1, x2, y2)`, where `(x1, y1)` is the top-left corner and `(x2, y2)` is the bottom-right corner of the rectangle.

Experiment with different coordinates to accurately cover the streets where vehicles exit. Visualize these ROIs by drawing rectangles on a frame of the video and adjust the coordinates as needed until you capture the desired areas effectively.

## Process Video Frames and Identify Vehicles Left the Road Form Each Street

For each frame in the video, use the YOLO model to detect and track vehicles. You will need to write a loop that processes each frame and applies the YOLO model.

In each frame, after detecting the vehicles, determine whether they exited the road via the top, bottom, left, or right street. You can use the positions of the bounding boxes provided by YOLO to do this.

- The video should display bounding boxes around the detected objects.
- The video should display your name on the top.
- It should display the calculated center of each vehicle.
- The video should display the confidence score, alongside the object ID and class ID of each detected and tracked object.
- The video should display the rectangles representing the regions where you count the vehicles that have crossed and exited.
- The video should display the number of vehicles that exited via the top street.
- The video should display the number of vehicles that exited via the bottom street.
- The video should display the number of vehicles that exited via the left street.
- The video should display the number of vehicles that exited via the right street.

# Detection,track and counting

In [19]:
import cv2
from ultralytics import YOLO

In [20]:
# Open the input video
video_path = '/content/YOLO_Video.mp4'
cap = cv2.VideoCapture(video_path)

In [21]:
width = int(cap.get(3))
height = int(cap.get(4))
fps = int(cap.get(5))

In [22]:
output_path = '/content/output_video.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

In [23]:
color = (0, 255, 0)
thickness = 2
font = cv2.FONT_HERSHEY_SIMPLEX
font_scale = 0.5
font_thickness = 1

ret = True
while ret:
    ret, frame = cap.read()

    if not ret:
        break

    # Initialize class counts
    class_counts = {}

    results = model.track(frame, persist=True)

    for result in results:
        for obj in result.boxes:
            bbox = obj.xyxy[0].cpu().numpy()  # Bounding box coordinates
            class_id = int(obj.cls[0].cpu().numpy()) if obj.cls is not None else -1  # Class ID
            conf = obj.conf[0].cpu().numpy() if obj.conf is not None else 0.0  # Confidence score
            id = int(obj.id[0].cpu().numpy()) if obj.id is not None else -1  # Unique ID

            # Update class count
            if class_id not in class_counts:
                class_counts[class_id] = 0
            class_counts[class_id] += 1

            # Convert bbox coordinates to integers
            x1, y1, x2, y2 = map(int, bbox)

            # Draw the bounding box
            cv2.rectangle(frame, (x1, y1), (x2, y2), color, thickness)

            # Prepare the label
            label = f'ID: {id} Class: {class_id} Conf: {conf:.2f}'

            # Put the label on the frame
            cv2.putText(frame, label, (x1, y1 - 10), font, font_scale, color, font_thickness, cv2.LINE_AA)

    # Draw class counts at the bottom of the frame
    text_offset = 20
    for class_id, count in class_counts.items():
        label = f'Class {class_id}: {count}'
        cv2.putText(frame, label, (10, height - text_offset), font, font_scale, color, font_thickness, cv2.LINE_AA)
        text_offset += 20  # Move to the next line

    out.write(frame)

cap.release()
out.release()

requirements: Ultralytics requirement ['lapx>=0.5.2'] not found, attempting AutoUpdate...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 41.7 MB/s eta 0:00:00

requirements: AutoUpdate success ✅ 3.5s, installed 1 package: ['lapx>=0.5.2']
requirements: ⚠️ Restart runtime or rerun command for updates to take effect


0: 384x640 2 buss, 82 cars, 2 trucks, 2 vans, 3980.5ms
Speed: 27.6ms preprocess, 3980.5ms inference, 32.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 buss, 82 cars, 2 trucks, 2 vans, 2753.1ms
Speed: 6.9ms preprocess, 2753.1ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 buss, 82 cars, 2 trucks, 2 vans, 2420.1ms
Speed: 3.6ms preprocess, 2420.1ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 buss, 82 cars, 2 trucks, 2 vans, 2455.2ms
Speed: 3.7ms preprocess, 2455.2ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 buss, 82 cars, 2 trucks, 2 vans, 2487.7m

In [25]:
#Calculted center of each vehical
center=c
center_x = (x1 + x2) // 2
center_y = (y1 + y2) // 2

NameError: name 'c' is not defined

## Save and Submit Your Work

In [ ]:
رابط الفيديو=https://drive.google.com/file/d/1iyvnGF46HPN6TF0cXKLhyvyQYbJByzEm/view?usp=drive_link

Submit both the notebook and the output video